# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from helper import run_inference, get_result_df
from azureml.core.run import Run
from azureml.core.model import Model


from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Dataset

### Overview
The Titanic dataset used in this project was downloaded from [Kaggle](https://www.kaggle.com/competitions/titanic/overview). We are provided a train and a test dataset with 891 and 418 records, respectively, with 12 columns.
From the given features for each record we want to make a prediction if this person did survive the Titanic disaster or not. Therefore relevant features should be identified to help us with this task.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
# Load workspace from config file present at .\config.json.
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

In [ ]:
# Choose a name for experiment
experiment_name = 'Titanic'
project_folder = './titanic-project'

experiment=Experiment(ws, experiment_name)
experiment

In [1]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
found = False
key = "Titanic_dataset"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        train_data = './data/train_modified.csv'
        dataset = Dataset.Tabular.from_delimited_files(train_data, separator=';')        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description="This is the complete dataset for the capstone project.")
        dataset_filtered = dataset.keep_columns(["Survived","Pclass","Sex","SibSp","Parch","Fare","Embarked","Age"])
        dataset = dataset.register(workspace=ws,
                                   name=key+"_filtered",
                                   description="This is the filtered dataset for the capstone project " \
                                   "with only those features relevant for training.")

df = dataset_filtered.to_pandas_dataframe()
df.describe()

NameError: name 'ws' is not defined

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

First, we create a compute cluster if it is not yet available. As we want to use deep learning, we choose a GPU for training.

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

num_nodes = 5

amlcompute_cluster_name = "ComputeClusterCapstone"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6',
                                                           vm_priority = 'lowpriority',
                                                           max_nodes=num_nodes)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

In [ ]:
automl_settings = {
    "max_concurrent_iterations": 5,
    "max_cores_per_iteration": -1,
    "enable_dnn": True,
    "enable_early_stopping": True,
    "validation_size": 0.2,
    "primary_metric" : 'accuracy',
    "enable_voting_ensemble": False,
    "enable_stack_ensemble": False
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset_filtered,
                             label_column_name="Survived",   
                             path = project_folder,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [ ]:
ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

In [ ]:
remote_run = experiment.submit(automl_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run = remote_run.get_best_child()

[Source](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/automated-machine-learning/classification-text-dnn/auto-ml-classification-text-dnn.ipynb)

In [ ]:
#TODO: Save the best model

# Download the featurization summary JSON file locally
best_run.download_file(
    "outputs/featurization_summary.json", "featurization_summary.json"
)

# Render the JSON as a pandas DataFrame
with open("featurization_summary.json", "r") as f:
    records = json.load(f)

featurization_summary = pd.DataFrame.from_records(records)
featurization_summary["Transformations"].tolist()

In [ ]:
summary_df = get_result_df(automl_run)
best_dnn_run_id = summary_df["run_id"].iloc[0]
best_dnn_run = Run(experiment, best_dnn_run_id)

In [ ]:
model_dir = "AutoML_model"  # Local folder where the model will be stored temporarily
if not os.path.isdir(model_dir):
    os.mkdir(model_dir)

best_dnn_run.download_file("outputs/model.pkl", model_dir + "/model.pkl")

In [ ]:
# Register the model
model_name = "Titanic_AutoML_model"
best_model = Model.register(
    model_path=model_dir + "/model.pkl", model_name=model_name, tags=None, workspace=ws
)

# Prediction
Predict values for `Survived` for the Kaggle competition.

In [ ]:
dataset_predict = Dataset.Tabular.from_delimited_files(path='./data/test_modified.csv')
dataset_predict_filtered = dataset_predict.keep_columns(["Pclass","Sex","SibSp","Parch","Fare","Embarked","Age"])
dataframe_predict_filtered = dataset_predict_filtered.to_pandas_dataframe()

y_pred = best_model.predict(dataframe_predict_filtered)

dataset_predict_output = dataset_predict.keep_columns(["PassengerId"])
dataframe_predict_output = dataset_predict_output.to_pandas_dataframe()
dataframe_predict_output = pd.concat([dataframe_predict_output, y_pred], axis=1)

dataframe_predict_output.to_csv("./data/test_predict.csv")
datastore = ws.get_default_datastore()

dataset_predict = Dataset.Tabular.register_pandas_dataframe(dataframe_predict_output,
                                                            target=datastore,
                                                            name=key+"_filtered",
                                                            description="These are the preditions for the test dataset."
                                                            )

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

[Source1](https://docs.microsoft.com/de-de/python/api/overview/azure/ml/?view=azure-ml-py)
[Source 2](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/deployment/deploy-to-cloud/model-register-and-deploy.ipynb)

TODO: In the cell below, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.webservice import Webservice

environment = Environment("Titanic-environment")

# Combine scoring script & environment in Inference configuration
inference_config = InferenceConfig(entry_script="score.py",
                                   environment=environment)

# Set deployment configuration
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1,
                                                       memory_gb = 1,
                                                       enable_app_insights=True)

# Define the model, inference, & deployment configuration and web service name and location to deploy
service = Model.deploy(workspace = ws,
                       name = "Titanic-webservice",
                       models = [best_model],
                       #inference_config = inference_config,
                       deployment_config = deployment_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import json

input_payload = json.dumps({
    'data': dataset_predict_filtered[0:2].tolist(),
    'method': 'predict'
})

output = service.run(input_payload)

print(output)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
!python logs.py

In [ ]:
# Delete the webservice
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
